<a href="https://colab.research.google.com/github/w4bo/teaching-bigdata/blob/main/notebooks/01-CaliforniaHousingPricing_v0_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# California housing pricing

This notebook runs on Google Colab. Colab provides a serverless Jupyter notebook environment for interactive development. (At the moment, 2022) Google Colab is free to use like other G Suite products.

In this laboratory we will build a simple data pipeline to get acquainted with the "main" steps necessary to transform your data.

The data contains information from the 1990 California census. It does provide an accessible introductory dataset for teaching people about the basics of machine learning. For simplcity, we will import the data from a csv file. Check also: https://www.kaggle.com/camnugent/california-housing-prices

From https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/

    This data has metrics such as the population, median income, median housing price, and so on for each block group in California.
    Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). We will just call them “districts” for short
    The goal is to build a model to predict the median housing price in any district, given all the other metrics

In case you need help with preprocessing with Pandas, check:

    https://github.com/w4bo/2022-bbs-dm/blob/main/notebooks/01-PandasFundaments.ipynb



First of all, we need to setup the Python environment by installing and importing the necessary Python dependencies.

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import seaborn as sns
import prov

print(pd.__version__)
print(sk.__version__)
print(np.__version__)
print(sns.__version__)
print(prov.__version__)


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
2.1.2
1.3.0
1.26.0
0.12.2
2.0.0


#### Data collection

Import the dataset. In this case, there is no need for ETL/integration since the dataset is ready for elaboration.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/w4bo/handsOnDataPipelines/main/01-MachineLearning/datasets/housing.csv")
df

HTTPError: HTTP Error 404: Not Found

#### Data understanding

Dataset description

1. `longitude`: A measure of how far west a house is; a higher value is farther west
2. `latitude`: A measure of how far north a house is; a higher value is farther north
3. `housingMedianAge`: Median age of a house within a block; a lower number is a newer building
4. `totalRooms`: Total number of rooms within a block
5. `totalBedrooms`: Total number of bedrooms within a block
6. `population`: Total number of people residing within a block
7. `households`: Total number of households, a group of people residing within a home unit, for a block
8. `medianIncome`: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. `medianHouseValue`: Median house value for households within a block (measured in US Dollars)
10. `oceanProximity`: Location of the house w.r.t ocean/sea

Let's take a look at the data

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='all')

#### Memory usage

What if I change float64 to float32?

In [ ]:
dff = df.copy(deep=True) # copy the dataframe
for x in df.columns: # iterate over the columns
    if dff[x].dtype == 'float64': # if the column has type `float64`
        dff[x] = dff[x].astype('float32') # ... change it to `float32`
dff.info() # show some statistics on the dataframe

There are some missing values in the column `total_bedorooms` what can we do?

Most Machine Learning algorithms cannot work with missing features. We have three options:

Get rid of the corresponding districts (i.e., drop the rows)

    df.dropna(subset=["total_bedrooms"])

Get rid of the whole attribute (i.e., drop the columns)

    df.drop("total_bedrooms", axis=1)

Set the values to some value (zero, the mean, the median, etc.)

    df["total_bedrooms"].fillna(df["total_bedrooms"].median())

#### Non-numeric attributes

`ocean_proximity` is a text attribute so we cannot compute its median. Some options:
- Get rid of the whole attribute. (`df.drop("ocean_proximity", axis=1`)
- Change from categorical to ordinal (e.g., `NEAR BAY` = 0, `INLAND` = 1)
    - Can foresee any problem in this?
    - ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g., for ordered categories such as “bad”, “average”, “good”, “excellent”), but it is obviously not the case for the ocean_proximity column (for example, categories 0 and 4 are clearly more similar than categories 0 and 1). 
- Change from categorical to one hot encoding
    - To fix this issue, a common solution is to create one binary attribute per category: one attribute equal to 1 when the category is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the category is “INLAND” (and 0 otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called dummy attributes

In [ ]:
df["ocean_proximity"].value_counts()

In [ ]:
df["ocean_proximity"].hist()

Change from categorical to ordinal

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
y = ordinal_encoder.fit_transform(df[["ocean_proximity"]])
y

From categorical to one-hot encoding

In [ ]:
y = pd.get_dummies(df["ocean_proximity"], prefix='ocean_proximity')
y

#### Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))
plt.show()

Open questions:

- `median_income` should be in dollars. However, it has a strange range. Why? "you are told that the data has been scaled and capped at 15 (actually 15.0001) for higher median incomes, and at 0.5 (actually 0.4999) for lower median incomes. The numbers represent roughly tens of thousands of dollars. The numbers represent roughly tens of thousands of dollars"
- `housing_median_age` and `median_house_value` are capped. As to `median_house_value`, this is a serious problem since it is your target attribute (your labels). Your Machine Learning algorithms may learn that prices never go beyond that limit. You need to check with your client team (the team that will use your system’s output) to see if this is a problem or not. If they tell you that they need precise predictions even beyond 500,000USD, then you have mainly two options: (a) collect proper labels for the districts whose labels were capped, (b) remove those districts from the training set."
- These attributes have very different scales. Should we scale them?
- Many histograms are tail heavy: they extend much farther to the right of the median than to the left. This may make it a bit harder for some Machine Learning algorithms to detect patterns

##### Are the relationships between variables?

Create a grid of Axes such that each numeric variable in data will by shared across the y-axes across a single row and the x-axes across a single column. The diagonal plots are treated differently: a univariate distribution plot is drawn to show the marginal distribution of the data in each column.

In [ ]:
tmp = df[["median_income", "housing_median_age", "median_house_value", "households", "population", "total_rooms"]]
# sns.pairplot(tmp.sample(n=1000, random_state=42), hue="median_house_value", markers='+')
sns.pairplot(tmp.sample(n=1000, random_state=42), markers='+')
plt.show()

#### Scaling attributes

Attributes have very different scales. Should we scale them?

- Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.
- Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.


##### Min-max normalization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.min()) / (num_df.max() - num_df.min())
normalized_df

##### Standardization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.mean()) / num_df.std()
normalized_df

#### Machine learning

In [ ]:
# For now we simply drop "ocean_proximity"
if "ocean_proximity" in df.columns:
  df = df.drop("ocean_proximity", axis=1)

# Let's create some dataset variation
# dataset1: drop the rows containing the null values and the columns `latitude` and `longitude` 
dataset_v1 = df.copy(deep=True).dropna().drop(["longitude", "latitude"], axis=1)

# dataset2: impute missing values with the average number of bedrooms 
dataset_v2 = df.copy(deep=True)
dataset_v2["total_bedrooms"] = dataset_v2["total_bedrooms"].fillna(dataset_v2["total_bedrooms"].mean())

# Create the list of datasets
datasets = [(dataset_v1, "datasetv1"), (dataset_v2, "datasetv2")]

In [ ]:
# Let's import some machine learning models (here we are not addressing hyper-parameter tuning)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

dt = DecisionTreeRegressor(random_state=0) # initialize decision tree regressor model
lr = LinearRegression() # initialize a linear regressor model

# Create the list of algorithms
ml_algorithms = [(lr, "lr"), (dt, "dt")]

In [ ]:
from sklearn.model_selection import cross_val_score

instances = []
i = 0

# For each dataset version...
for dataset, dataset_version in datasets:
  # Get the feature matrix
  X = dataset.drop(columns=["median_house_value"]).to_numpy()
  # Get the train label array
  y = dataset["median_house_value"].to_numpy() 
  
  # For each machine learning algorithm...
  for ml_algorithm, ml_algorithm_version in ml_algorithms:
    # Run the machine learning algorithm on the given dataset
    # Each run is a pipeline instance that we shoul compare against the others
    instance = {}
    instance["id"] = i  # store the id of the instance 
    instance["dataset"] = dataset_version  # store the version of the dataset
    instance["algorithm"] = ml_algorithm_version  # store the version of the ml algorithm
    instance["score"] = cross_val_score(ml_algorithm, X, y, cv=10).mean()  # store the performance of the pipeline instance
    instances = instances + [instance]
    i += 1

# Collect the results
result = pd.DataFrame.from_dict(instances, orient='columns')
sns.catplot(x = "dataset", y = "score", hue = "algorithm", data = result, kind = "bar")

And what about data provenance?

In [ ]:
from prov.model import ProvDocument
from prov.dot import prov_to_dot
from IPython.display import Image

# Create a new provenance document
d1 = ProvDocument()  # d1 is now an empty provenance document
d1.add_namespace('unibo', 'https://www.unibo.it')  # add the namespace
d1.add_namespace('sk', 'https://scikit-learn.org/stable/')  # add the namespace
agent = d1.agent('unibo:mfrancia')  # add an agent

# For each dataset version...
for dataset, dataset_version in datasets:
  original_dataset = d1.entity('unibo:' + dataset_version)  # register the dataset
  d1.wasAttributedTo(original_dataset, agent)  # attribute the dataset to the agent who created it
  # For each machine learning algorithm...
  for ml_algorithm, ml_algorithm_version in ml_algorithms:
    algo = d1.activity('sk:' + ml_algorithm_version)  # register the algorithm as a (processing) activity
    processed_dataset = d1.entity('unibo:' + ml_algorithm_version + "_" + dataset_version, {'sk:cv-score': '...'})  # create an activity represented the processed dataset
    d1.used(algo, original_dataset)  # the activity used the dataset as input
    d1.wasGeneratedBy(processed_dataset, algo)  # the processed dataset has been created by the algorithm
    d1.wasDerivedFrom(processed_dataset, original_dataset)  # the processed dataset has been derived from the original one

# visualize the graph
dot = prov_to_dot(d1)
dot.write_png('prov.png')
Image('prov.png')

#### Hands on!

##### Data pre-processing

In [ ]:
num_df = df.copy(deep=True) # do not change this line

# Filling in (i.e., impute) missing values with the median value 
num_df["total_bedrooms"] = 1 # change `1` with the proper solution 

# Add a new column: population_per_household = population / households
num_df["population_per_household"] = 1 # change `1` with the proper solution 

# Add a new column: rooms_per_household = total_rooms / households
num_df["rooms_per_household"] = 1 # change `1` with the proper solution 

# Add a new column: bedrooms_per_room = total_bedrooms / total_rooms
num_df["bedrooms_per_room"] = 1 # change `1` with the proper solution 

# Apply standardization to all the numeric columns
num_df = pd.DataFrame() # change `pd.DataFrame()` with the proper solution 

# One hot encode `ocean_proximity` since it is a categorical attribute 
cat_df = pd.DataFrame() # change `pd.DataFrame()` with the proper solution (hint: pd.get_dummies)

clean_df = pd.concat([num_df, cat_df], axis=1) # do not change this line
clean_df

##### Machine learning

Can you build a better model?

In [ ]:
# Try your sk-learn model here